In [4]:
from llama_index.llms.ollama import Ollama
model_name="llama3.1"

In [8]:
llm = Ollama(model=model_name, request_timeout=120.0)
user_question = input("Please enter your query for LLM: ")
print("Generating output ...")
resp = llm.complete(user_question)
print(resp)

Please enter your query for LLM. What is vedic astrology?


Generating output ...
Vedic astrology, also known as Jyotish or Hindu astrology, is an ancient system of astrology that originated in India and is based on the principles of the Vedas, a collection of sacred texts from the Indian subcontinent. It is considered one of the oldest forms of astrology still practiced today.

Here are some key aspects of Vedic astrology:

1. **Based on ancient texts**: Vedic astrology is rooted in the Vedas, which were composed between 1500 BCE and 500 BCE.
2. **Astrological system**: Jyotish is a complete system that includes both predictive (foretelling future events) and prescriptive (providing guidance for personal growth) aspects of life.
3. **Twelve zodiac signs**: Vedic astrology uses the same twelve zodiac signs as Western astrology: Aries, Taurus, Gemini, Cancer, Leo, Virgo, Libra, Scorpio, Sagittarius, Capricorn, Aquarius, and Pisces.
4. **However, with different planetary associations**: In Vedic astrology, each sign is associated with a specific 

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from openai import OpenAI
import instructor

class Character(BaseModel):
    name: str
    age: int
    fact: List[str] = Field(..., description="A list of facts about the character")


# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="llama3.1",
    messages=[
        {
            "role": "user",
            "content": "Tell me about Shah Rukh Khan",
        }
    ],
    response_model=Character,
)
print(resp.model_dump_json(indent=2))

# Classification System

In [15]:
from enum import Enum

ticket1 = """
I ordered a laptop from your store last week (Order #12345), but I received a tablet instead. 
This is unacceptable! I need the laptop for work urgently. Please resolve this immediately or I'll have to dispute the charge.
"""

ticket2 = """
Hello, I'm having trouble logging into my account. I've tried resetting my password, but I'm not receiving the reset email. 
Can you please help me regain access to my account? I've been a loyal customer for years and have several pending orders.
"""

ticket3 = """
Kya faaltu website hai. My laptop conked off and no one picks the phone.
"""

class TicketCategory(str, Enum):
    ORDER_ISSUE = "order_issue"
    ACCOUNT_ACCESS = "account_access"
    PRODUCT_INQUIRY = "product_inquiry"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    OTHER = "other"

class CustomerSentiment(str, Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"
    SATISFIED = "satisfied"

class TicketUrgency(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class TicketClassification(BaseModel):
    category: TicketCategory
    urgency: TicketUrgency
    sentiment: CustomerSentiment
    confidence: float = Field(ge=0, le=1, description="Confidence score for the classification")
    key_information: List[str] = Field(description="List of key points extracted from the ticket")
    suggested_action: str = Field(description="Brief suggestion for handling the ticket")
    
    
ticket_classification = TicketClassification(
    category=TicketCategory.ORDER_ISSUE,
    urgency=TicketUrgency.HIGH,
    sentiment=CustomerSentiment.ANGRY,
    confidence=0.9,
    key_information=["Order #12345", "Received tablet instead of laptop"],
    suggested_action="Contact customer to arrange laptop delivery"
)

In [16]:
def classify_ticket(ticket_text: str) -> TicketClassification:

    response = client.chat.completions.create(
        model="llama3.1",
        response_model=TicketClassification,
        temperature=0,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": "Analyze the following customer support ticket and extract the requested information."
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

result1 = classify_ticket(ticket1)
result2 = classify_ticket(ticket2)
result3 = classify_ticket(ticket3)


print(result1.model_dump_json(indent=2))
print(result2.model_dump_json(indent=2))
print(result3.model_dump_json(indent=2))

{
  "category": "order_issue",
  "urgency": "high",
  "sentiment": "angry",
  "confidence": 0.9,
  "key_information": [
    "Order #12345",
    "Received tablet instead of laptop"
  ],
  "suggested_action": "Replace laptop with correct order or provide immediate refund"
}
{
  "category": "account_access",
  "urgency": "high",
  "sentiment": "frustrated",
  "confidence": 0.8,
  "key_information": [
    "pending_orders",
    "reset_password_issue"
  ],
  "suggested_action": "escalate_ticket_to_support_team"
}
{
  "category": "technical_support",
  "urgency": "high",
  "sentiment": "frustrated",
  "confidence": 0.8,
  "key_information": [
    "laptop conked off",
    "no one picks the phone"
  ],
  "suggested_action": "Contact customer support via email or live chat for assistance"
}


In [23]:
SYSTEM_PROMPT = """
You are an AI assistant for a large e-commerce platform's customer support team. 
Your role is to analyze incoming customer support tickets and provide structured information to help our team respond quickly and effectively.
Business Context:
- We handle thousands of tickets daily across various categories (orders, accounts, products, technical issues, billing).
- Quick and accurate classification is crucial for customer satisfaction and operational efficiency.
- We prioritize based on urgency and customer sentiment.
Your tasks:
1. Categorize the ticket into the most appropriate category.
2. Assess the urgency of the issue (low, medium, high, critical).
3. Determine the customer's sentiment.
4. Extract key information that would be helpful for our support team.
5. Suggest an initial action for handling the ticket.
6. Provide a confidence score for your classification.
Remember:
- Be objective and base your analysis solely on the information provided in the ticket.
- If you're unsure about any aspect, reflect that in your confidence score.
- For 'key_information', extract specific details like order numbers, product names, or account issues.
- The 'suggested_action' should be a brief, actionable step for our support team.
Analyze the following customer support ticket and provide the requested information in the specified format.
"""

def classify_ticket(ticket_text: str) -> TicketClassification:

    response = client.chat.completions.create(
        model="llama3.1",
        response_model=TicketClassification,
        temperature=1,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

result1 = classify_ticket(ticket1)
result2 = classify_ticket(ticket2)
result3 = classify_ticket(ticket3)


print(result1.model_dump_json(indent=2))
print(result2.model_dump_json(indent=2))
print(result3.model_dump_json(indent=2))

{
  "category": "order_issue",
  "urgency": "critical",
  "sentiment": "angry",
  "confidence": 1.0,
  "key_information": [
    "Order #12345",
    "Received tablet instead of laptop"
  ],
  "suggested_action": "Process immediate replacement or refund and contact customer for resolution."
}
{
  "category": "account_access",
  "urgency": "high",
  "sentiment": "frustrated",
  "confidence": 0.8,
  "key_information": [
    "pending orders",
    "unable to reset password"
  ],
  "suggested_action": "Escalate ticket to account access team for further assistance and attempt to reset password again or look up previous login history."
}
{
  "category": "technical_support",
  "urgency": "high",
  "sentiment": "frustrated",
  "confidence": 0.8,
  "key_information": [
    "laptop conked off",
    "no support from website"
  ],
  "suggested_action": "Please escalate the issue to our technical team and provide a phone number or email for contact."
}
